## 1. Install and import dependencies

In [5]:
import sys
!{sys.executable} -m pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [2]:
import sys
!{sys.executable} -m pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## 2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

C:\Users\Anwender\sentiment_analysis\sentiment_analysis\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Anwender\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## 3. Encode and calculate Sentiment

In [14]:
tokens = tokenizer.encode('It was not really nice what i ate', return_tensors='pt')

In [15]:
result = model(tokens)

In [16]:
result.logits

tensor([[ 0.7386,  2.2746,  1.9819, -0.9676, -3.2683]],
       grad_fn=<AddmmBackward0>)

In [17]:
int(torch.argmax(result.logits))+1

2

## 4. Collect reviews

In [19]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')

In [20]:
soup = BeautifulSoup(r.text,'html.parser')

In [21]:
regex = re.compile('.*comment.*')

In [22]:
results = soup.find_all('p',{'class':regex})

In [23]:
reviews = [result.text for result in results]

In [24]:
reviews

['The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!',
 'Visiting from Texas and decided to give this restaurant a try. \xa0We were pleasantly surprised. \xa0While the margaritas are more like martinis, the food was excellent. \xa0More like a tapas, Mexican fusion. \xa0Great way to try different plates.The real treat was Chelsea, our waitress. \xa0Took the time to explain the menu and offer suggestions. \xa0Always smiling and very pleasant. \xa0Best service we have had in Sydney!',
 'Great atmosphere, attentive service, solid margs, and a Tasty menu. The Brisket Tacos were substantial and delicious. The corn ribs??? \xa0Fawgetaboutit! \xa0Unreal. \xa0Wanted to order another plate.',
 "Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the famili

## 5. Load Reviews into Dataframe and score

In [25]:
import numpy as np
import pandas as pd

In [27]:
np.array(reviews)

array(['The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!',
       'Visiting from Texas and decided to give this restaurant a try. \xa0We were pleasantly surprised. \xa0While the margaritas are more like martinis, the food was excellent. \xa0More like a tapas, Mexican fusion. \xa0Great way to try different plates.The real treat was Chelsea, our waitress. \xa0Took the time to explain the menu and offer suggestions. \xa0Always smiling and very pleasant. \xa0Best service we have had in Sydney!',
       'Great atmosphere, attentive service, solid margs, and a Tasty menu. The Brisket Tacos were substantial and delicious. The corn ribs??? \xa0Fawgetaboutit! \xa0Unreal. \xa0Wanted to order another plate.',
       "Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the foo

In [29]:
df = pd.DataFrame(np.array(reviews), columns=['Reviews'])

In [30]:
df

,Reviews
0,The food and service here was really good. It...
1,Visiting from Texas and decided to give this r...
2,"Great atmosphere, attentive service, solid mar..."
3,Don't come here expecting legit Mexican food b...
4,Out of all the restaurants that I tried in Syd...
5,We came here on a Thursday night @ 5pm and by ...
6,The food is fresh and tasty. The scallop cevi...
7,Have been here twice and have absolutely loved...
8,I was pleasantly surprised at what a great job...
9,If you're looking for a quiet little romantic ...


In [32]:
df['Reviews'].iloc[1]

'Visiting from Texas and decided to give this restaurant a try. \xa0We were pleasantly surprised. \xa0While the margaritas are more like martinis, the food was excellent. \xa0More like a tapas, Mexican fusion. \xa0Great way to try different plates.The real treat was Chelsea, our waitress. \xa0Took the time to explain the menu and offer suggestions. \xa0Always smiling and very pleasant. \xa0Best service we have had in Sydney!'

In [33]:
def sentiment_score(review):
    token = tokenizer.encode(review,return_tensors='pt')
    result= model(token)
    return int(torch.argmax(result.logits))+1
    

In [34]:
df['sentiment_score']= df['Reviews'].apply(lambda x: sentiment_score(x[:512]))

In [35]:
df

,Reviews,sentiment_score
0,The food and service here was really good. It...,5
1,Visiting from Texas and decided to give this r...,5
2,"Great atmosphere, attentive service, solid mar...",3
3,Don't come here expecting legit Mexican food b...,3
4,Out of all the restaurants that I tried in Syd...,5
5,We came here on a Thursday night @ 5pm and by ...,4
6,The food is fresh and tasty. The scallop cevi...,4
7,Have been here twice and have absolutely loved...,5
8,I was pleasantly surprised at what a great job...,5
9,If you're looking for a quiet little romantic ...,2


In [36]:
df['Reviews'].iloc[9]

"If you're looking for a quiet little romantic place, this is not it. Lots of hard surfaces create a very noisy atmosphere. But, if you're looking for a new age take on Mexican food, you're in for some very interesting dishes.We ordered the salmon tacos, tamales, blackened chicken, and watermelon salad.The salmon tacos were delicious, but points off for cold flour tortillas instead of fresh hot corn. (No choice was offered). \xa0The tamales were unrecognizable by Mexican standards, but still very tasty. The blackened chicken was good but it would've been nice if some tortillas would have been offered which is customary in most Mexican restaurants. The watermelon salad was a taste sensation, even though it had a bit too much cilantro for my taste. My wife absolutely loved it. \xa0The plantain chips were very good and the guacamole was some of the best we've had.Our server was attentive but not overbearing. She even came to apologize that the food was not timed appropriately and offered 